# Exercise #2

This notebook is created by **Chinchuthakun Worameth** as a part of **Sparse Signal Processing and Optimization (ART.T465)** at Tokyo Institute of Technology taught in Fall semester 2021 by Prof. Ono Shunsuke. It contains
- Implementation of proximal operator for the L1 norm and the nuclear norm
- Implementation of projection onto the box contraint
Symbol **[Qx]** indicates that a section or subsection corresponds to the xth question in the lecture slide.

## 1. Proximal Operator

**Proximal operator** of function $f$ is defined as
$$
\text{prox}_{\gamma f}(x) = \text{argmin}_y f(y) + \frac{1}{2\gamma}\Vert x - y \Vert^2, \gamma>0
$$

### 1.1. L1 norm
We define **L1 norm** and its proximal operator as
$$
f(x) = \Vert x \Vert_1 \rightarrow \text{prox}_{\gamma f}(x) = \text{SoftThre}(x,\gamma)
$$
such that
$$
\text{SoftThre}(x,\gamma) = \text{sgn}(x) \circ \text{max}(\text{abs}(x), 0)
$$
where $\text{sgn}(x)$, $\circ$, and $\text{abs}(x)$ are the sign function, Hadamard product, and element-wise absolute value of $x$ respectively.

### 1.2. Nuclear norm
Similarly, **Nuclear norm** and its proximal operator are
$$
g(X) = \Vert X \Vert_* \rightarrow \text{prox}_{\gamma g}(X) = U(\text{SoftThre}(\Sigma, \gamma))V^T
$$
where $X=U\Sigma V^T$, i.e. **Singular Value Decomposition (SVD)**.

## 2. Projection



**Convex projection** of $x$ onto a closed convex set $C$ is defined as
$$
P_C(x) = \text{argmin}_{y \in C}\Vert x - y\Vert
$$
Using the indicator function $1_C$ to map points inside and outside $C$ to $0$ and $\infty$, it is obvious that
$$
P_C(x) = \text{prox}_{\gamma 1_C}(x)
$$

### 2.1. Box constraint
Given $C = [a,b]^N$ such that $a < b$, by using the indicator function, we can define the projection onto the box constraint as
$$
P_C(x) = [\max(a,\min(b,x_i))]_i
$$

## 3. Implementation

We define class ```proximalOperator``` for L1 norm, Nuclear norm, and box constraint (since convex projection is equivalent to proximal operator). Then, we demonstrate them on random inputs.

In [ ]:
import numpy as np
import random

In [ ]:
class proximalOperator:
    def __init__(self, param):
        self.param = param
    
    def softThreshold(self, x):
        return np.sign(x) * np.maximum(np.absolute(x) - self.param['gamma'], np.zeros(x.shape))
    
    def calculate(self, x):
        if self.param['name'] == "l1":
            return self.softThreshold(x)
        elif self.param['name'] == "nuclear":
            """
            np.linalg.svd returns s as a vector [s_1, ..., s_r]
            So, we need to convert it to a diagonal matrix first.
            """
            u, s, vt = np.linalg.svd(x)
            s_full = np.zeros(x.shape)
            s_full[:s.shape[0], :s.shape[0]] = np.diag(s)
            return np.matmul(np.matmul(u,self.softThreshold(s_full)),vt)
        elif self.param['name'] == "box":
            all_a = np.full(x.shape, self.param['a'])
            all_b = np.full(x.shape, self.param['b'])
            return np.maximum(all_a, np.minimum(x, all_b))

### 3.1. L1 norm [Q1]

In [ ]:
param = dict(name = "l1", gamma = 0.5)
proxOp = proximalOperator(param)

In [ ]:
# input is a vector
x = np.random.rand(5)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))
print("#"*20)
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [0.25352466 0.86344443 0.88904546 0.07573579 0.04068494]
Prox:  [0.         0.36344443 0.38904546 0.         0.        ]
####################
Input:  [[0.63836989 0.5866354  0.56982394 0.2799944  0.36248307 0.82815895]
 [0.28863846 0.9899887  0.79943726 0.71339478 0.28323414 0.86946948]
 [0.36777962 0.64656872 0.32130778 0.68209535 0.3214527  0.5467438 ]
 [0.27950722 0.89512642 0.13536162 0.59051664 0.30407004 0.62240308]
 [0.33945157 0.46871635 0.38897496 0.5735065  0.16166209 0.26709962]]
Prox:  [[0.13836989 0.0866354  0.06982394 0.         0.         0.32815895]
 [0.         0.4899887  0.29943726 0.21339478 0.         0.36946948]
 [0.         0.14656872 0.         0.18209535 0.         0.0467438 ]
 [0.         0.39512642 0.         0.09051664 0.         0.12240308]
 [0.         0.         0.         0.0735065  0.         0.        ]]


### 3.2. Nuclear norm [Q2]

In [ ]:
param = dict(name = "nuclear", gamma = 0.5)
proxOp = proximalOperator(param)

In [ ]:
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [[0.5070483  0.05779902 0.98476457 0.76142711 0.6371538  0.34244772]
 [0.76130478 0.1239488  0.14486878 0.59586812 0.95019852 0.53247865]
 [0.12873077 0.51916615 0.16243069 0.28861336 0.26092181 0.44491663]
 [0.84951619 0.32050684 0.01921326 0.21998819 0.48307735 0.86398859]
 [0.41206492 0.82133507 0.83653565 0.33447185 0.53786951 0.14003848]]
Prox:  [[0.45334216 0.22799223 0.62956494 0.51879056 0.57776861 0.2833454 ]
 [0.60191029 0.18394152 0.27521721 0.43327405 0.60007088 0.50956331]
 [0.25659275 0.26231736 0.1897469  0.16931423 0.25616613 0.23732877]
 [0.55960391 0.25406052 0.08916001 0.29285433 0.49332375 0.53961485]
 [0.34210656 0.45897766 0.59052552 0.36874676 0.43735308 0.24449153]]


### 3.3. Projection onto Box constraint [Q3]

In [ ]:
param = dict(name = "box", a = .5, b = .8)
proxOp = proximalOperator(param)

In [ ]:
# input is a vector
x = np.random.rand(5)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))
print("#"*20)
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [0.135674   0.4312398  0.99308087 0.44355151 0.19559666]
Prox:  [0.5 0.5 0.8 0.5 0.5]
####################
Input:  [[0.74717634 0.20043676 0.83813273 0.81086173 0.02033078 0.45770066]
 [0.25149955 0.19875    0.88730641 0.58821065 0.99607698 0.90066226]
 [0.43694019 0.5521144  0.36866127 0.59261047 0.20742795 0.03302646]
 [0.17539294 0.27923218 0.64947372 0.75859171 0.43583133 0.32755492]
 [0.48124378 0.51225917 0.62621853 0.78518231 0.84620731 0.51896223]]
Prox:  [[0.74717634 0.5        0.8        0.8        0.5        0.5       ]
 [0.5        0.5        0.8        0.58821065 0.8        0.8       ]
 [0.5        0.5521144  0.5        0.59261047 0.5        0.5       ]
 [0.5        0.5        0.64947372 0.75859171 0.5        0.5       ]
 [0.5        0.51225917 0.62621853 0.78518231 0.8        0.51896223]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ceb3b963-7d26-4b9f-840e-3ecd7c45577b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>